<a href="https://colab.research.google.com/github/t00199360/AI_Assignment2-2/blob/master/Draughts_Start.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class CurrentBoard:
  def __init__(self, setup_of_board = ["wwww", "wwww", "wwww", "    ", "    ", "bbbb", "bbbb", "bbbb"]):
      self.board = setup_of_board
      self.state = self.state_of_play()

  def display_board(self):
    for row in range(8):
      c = self.board[row]

      print("-----------------")
      if (row % 2) == 0:
        print("| |{}| |{}| |{}| |{}|".format(c[0], c[1], c[2], c[3]))
      else:
        print("|{}| |{}| |{}| |{}| |".format(c[0], c[1], c[2], c[3]))

  def all_possible_moves_for_position(self, position, inc):
    # inc = 1 for white, inc = -1 for black, inc = 0 for king
    all_moves = []

    if inc == 0:
      return self.all_possible_moves_for_position(position, -1) + self.all_possible_moves_for_position(position, 1)

    start_row = position[0]
    start_i = position[1]

    new_row = start_row + inc
    if (new_row >= 0) and (new_row < 8):
      all_moves.append((new_row, start_i))
      if (start_row % 2) == 0:
        if start_i < 3:
          all_moves.append((new_row, start_i + 1))
      else:
        if start_i > 0:
          all_moves.append((new_row, start_i - 1))

    return all_moves

  def playing_as(self, piece):
    if piece == "W":
      return "w"
    if piece == "B":
      return "b"

    return piece

  def all_possible_moves_for(self, _this):  # all_possile_moves_for "w" or "b"
    all_boards = []
    
    for row in range(8):
      for i in range(4):
        if self.playing_as(self.board[row][i]) == _this:
          inc = 0
          if self.board[row][i] == _this:
            if _this == "w":
              inc = 1
            else:
              inc = -1
          all_boards += self.construct_boards_for((row, i), self.all_possible_moves_for_position((row, i), inc))

    return all_boards

  def get_piece_at(self, position):
    return self.board[position[0]][position[1]]

  def row_for(self, position):
    return position[0]

  def i_for(self, position):
    return position[1]

  def replace_at(self, row_as_string, index, character):
    return row_as_string[:index] + character + row_as_string[index + 1:]

  def jump_to(self, from_position, over_position):
    row_dir = self.row_for(over_position) - self.row_for(from_position)
    new_row = self.row_for(over_position) + row_dir

    if (new_row < 0) or (new_row > 7):
      return None

    if (new_row % 2) == 0:
      if self.i_for(from_position) == self.i_for(over_position):
        new_i = self.i_for(from_position) - 1
      else:
        new_i = self.i_for(from_position) + 1
    else:
      if self.i_for(from_position) == self.i_for(over_position):
        new_i = self.i_for(from_position) + 1
      else:
        new_i = self.i_for(from_position) - 1

    if (new_i >= 0) and (new_i < 4):
      return (new_row, new_i)

  def all_kills(self, start_position, current_position, kill_list):
    all_kills_from_here = self.get_adjacent_kills(current_position, kill_list, self.get_piece_at(start_position))
    all_completed_moves = []

    if all_kills_from_here == []:
      if kill_list == []:
        return[]
      else:
        return [self.board_for(start_position, current_position, kill_list)]

    for kill in all_kills_from_here:
      all_completed_moves += self.all_kills(start_position, self.jump_to(current_position, kill), kill_list + [kill])

    return all_completed_moves

  def get_adjacent_kills(self, position, kill_list, piece):
    inc = 0
    if self.board[row][i] == _this:
      if _this == "w":
        inc = 1
      else:
        inc = -1

    kills = []
    all_adjecent_squares = self.all_possible_move_for_position(position, inc)

    for square in all_adjecent_squares:
      if self.is_opponent_of(piece, get_piece_at(square)):
        if self.get_piece_at(self.jump_to(position, square)) == " ":
          kills.append(square)

    return kills

  def is_opponent_of(self, piece1, piece2):
    return (self.playing_as(piece1) != self.laying_as(piece2)) and (piece1 != " ") and (piece2 != " ")

  def move_piece_to(self, from_position, to_position):
    board_def = []
    piece = self.get_piece_at(from_position)

    for row_index in range(8):
      row = self.board[row_index]
      if row_index == self.row_for(from_position):
        board_def.append(self.replace_at(row, self.i_for(from_position), " "))
      elif row_index == self.row_for(to_position):
        board_def.append(self.replace_at(row, self.i_for(to_position), piece))
      else:
        board_def.append(row)

    return CurrentBoard(board_def)

  def construct_boards_for(self, from_position, to_position_list):
    boards = []
                       
    for new_position in to_position_list:
      if self.get_piece_at(new_position) == " ":
        boards.append(self.move_piece_to(from_position, new_position))
    return boards

  def state_of_play(self):

    '''#012
    #345
    #678
    if self.equal3(self.board[0],self.board[1],self.board[2] ) or self.equal3(self.board[0],self.board[3],self.board[6] ) or self.equal3(self.board[0],self.board[4],self.board[8] ):
      if self.board[0] != " ":
        return self.board[0]

    if self.equal3(self.board[3],self.board[4],self.board[5]  ) or  self.equal3(self.board[4],self.board[1],self.board[7] ) or self.equal3(self.board[4],self.board[6],self.board[2] ):
      if self.board[4] != " ":
        return self.board[4]

    if self.equal3(self.board[6],self.board[7],self.board[8]  ) or  self.equal3(self.board[2],self.board[5],self.board[8] ) :
      if self.board[8] != " ":
        return self.board[8]  
      

    if " " in self.board:
      return "U"

    return "D"'''
    return "x"

In [ ]:
q = CurrentBoard()
q.display_board()

-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| |w|
-----------------
| | | | | | | | |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |


In [ ]:
q.all_possible_moves_for_position((7, 0), 0)

[(6, 0)]

In [ ]:
moves = q.all_possible_moves_for("w")

for m in moves:
  m.display_board()
  print("\n\n")

-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| | | |w| |w| |w|
-----------------
|w| | | | | | | |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |



-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| | | |w| |w| |w|
-----------------
| | |w| | | | | |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |



-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| | | |w| |w|
-----------------
| | |w| | | | | |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |



-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| | | |w| |w|
-----------------
|

In [ ]:
q.jump_to((1, 2), (0,1))

In [ ]:
class Search_Tree_Node:
  def __init__(self, game , playing_char, ply = 0):
    self.children =[]
    self.ply_depth = ply
    self.value_is_assigned = False
    self.value = 999
    self.board_for_node = game
    self.playing = playing_char
    if self.board_for_node.state == "U":
      self.generate_children()
    else:
      if self.board_for_node.state == "D":
        self.value = 0
      else: 
        if (self.ply_depth % 2) ==0:
          self.value = -1
        else:
          self.value = 1
      self.value_is_assigned = True
      



  def max_min_value(self):
    # if no value is assigned we will assign the max(or min) of the value of the children

    if  self.value_is_assigned:
        return self.value

    else: 
      self.children = sorted(self.children, key= lambda x: x.max_min_value() )

      if (self.ply_depth % 2) == 0:
        self.value = self.children[-1].value
      else:
        self.value = self.children[0].value
    return self.value


  def other_char(self,current):
    if current == "X":
      return "O"
    if current == "O":
      return "X"
    return " "

  def generate_children(self):
    for next_move in self.board_for_node.all_possible_moves_for(self.playing):
      self.children.append(Search_Tree_Node(next_move, self.other_char(self.playing), self.ply_depth + 1))



In [ ]:
s = Search_Tree_Node(CurrentBoard(),"X")

In [ ]:
s.max_min_value()

-1

In [ ]:
def run_game():
  current_game = CurrentBoard()
  #  Ask player do they want to move first?
  choice = input("Do you want to go first y/n?")
  player_going_first = (choice == "y") or (choice == "Y")

  player_this_turn = player_going_first


  #  Do you want X or O?

  choice = input("Do you want to play X's  y/n?")
  player_playing_X = (choice == "y") or (choice == "Y")
  if player_playing_X:
    player_playing = "X"
    computer_playing = "O"
  else:
    player_playing = "O"
    computer_playing = "X"
  #  Loop
  for i in range(9):

   # Display Current board (for player input)  
    current_game.display_board(game_display = True)
    if player_this_turn:
      print(" ")
      player_move = input("Choose your move --")
      index_of_player_move = int(player_move)


      new_board_string = current_game.board[:index_of_player_move] + player_playing + current_game.board[index_of_player_move+1:]
      current_game= CurrentBoard(new_board_string)
    else:
      print(" ")
      st = Search_Tree_Node(current_game,computer_playing)
      print(st.max_min_value())
      current_game =  st.children[-1].board_for_node

    player_this_turn = not  player_this_turn

    if current_game.state !="U":
      if current_game.state == "D":
        print("I cant believe it you drew with me!!!")
      elif computer_playing == current_game.state:
        print(" I win too easy !!!")
      else:
        print("The world is coming to an end!!!")
      current_game.display_board()
      break
   
  


In [ ]:
run_game()

KeyboardInterrupt: ignored